In [2]:


#pdf plumber
#csv.preview
import pandas as pd
#import unicode character database
import unicodedata
#import regular expression operations
import re

#import natural language toolkit
import nltk


#import our aquire
from acquire import *


#import our stopwords list
from nltk.corpus import stopwords
from copy import deepcopy




original = "Paul Erdős and George Pólya were influential Hungarian mathematicians who contributed \
a lot to the field. Erdős's name contains the Hungarian letter 'ő' ('o' with double acute accent), \
but is often incorrectly written as Erdos or Erdös either by mistake or out of typographical necessity"





def split_apply_join(funct,listobj):
    'helperfuction letters'

    mapped=map(funct, listobj)
    mapped=list(mapped)
    mapped=''.join(mapped)
  
    return mapped






def stopfilter(text,stop_words_extend_reduce=["'"]):
    'we use symmetric difference so if a is already in stop words then it will be added to our third set else our third set will be missing it'
    #create oujr english stopwords list

    stops = set(stopwords.words('english'))
    stop_words_extend_reduce=set(stop_words_extend_reduce)
    stops=stops.symmetric_difference(stop_words_extend_reduce)

    # stops=(stops|stop_words_extend)-exclude_words
    #another way
    
    filtered=list(filter((lambda x: x not in stops), text.split()))
    filtered=' '.join(filtered)

    return filtered









def basic_clean(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''   
    Filters out all special characters if you need to edit then supply a new regex filter 
    
    
    
    
    '''
    #make a copy and begin to transform it
    newtext = text.lower()

    #encode into ascii then decode
    newtext = unicodedata.normalize('NFKD', newtext)\
    .encode('ascii', 'ignore')\
    .decode('utf-8')

    #use re.sub to remove special characters
    newtext = re.sub(fr'{regexfilter}', '', newtext)

    return newtext

 


def tokenizer(text,regexfilter=r'[^a-z0-9\'\s]'):
    ''' 
    For a large file just save it locally
    
    
    
    
    
    '''
    newtext=basic_clean(text,regexfilter=regexfilter)
    #make ready tokenizer object
    tokenize = nltk.tokenize.ToktokTokenizer()
    #use the tokenizer
    newtext = tokenize.tokenize(newtext, return_str=True)
    return newtext





    
    
    
    


def stemmed(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    Takes text, tokenizes it, stems it
    
    
    
    
    
    '''
    #make ready porter stemmer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    ps = nltk.porter.PorterStemmer()
    stemmedlist=split_apply_join(ps.stem,newtext)
    return stemmedlist




def lemmatizor(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    
      Takes text, tokenizes it, lemmatizes it
    
    
    
    
    
    '''

    #make ready the lemmatizer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized=split_apply_join(wnl.lemmatize,newtext)
    return lemmatized



def dictlist_lemmatizor_else_stemmer(dictlistog,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"],lemmatize=True):
    ''' 
        
        
        
        
        #   Stem or Lemmatize 
        # if file is large and resources are scarce stem otherwise you can expect better performace from lemmatize
    '''
    dictlist=deepcopy(dictlistog)
    if lemmatize==True:
        for i in range(0,len(dictlist)):
            keys=dictlist[i].keys()
            for k in keys:
                text=dictlist[i].get(k)
                lemmatized=lemmatizor(text,regexfilter)
                stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=stop_words_extend_reduce)
                dictlist[i].update({k:stopfilteredlemitezed})
    else:
        for i in range(0,len(dictlist)):
            keys=dictlist[i].keys()
            for k in keys:
                text=dictlist[i].get(k)
                stem=stemmed(text,regexfilter)
                stopfilteredstem=stopfilter(stem,stop_words_extend_reduce=stop_words_extend_reduce)
                dictlist[i].update({k:stopfilteredstem})
    return dictlist



def dictlist_super_NLP_comp(dictlist,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"],interestingkeys=[]):
    ''
    ndictlist=deepcopy(dictlist)
    for i in range(0,len(ndictlist)):            
            
            for a in range(0,len(interestingkeys)):
                k=interestingkeys[a]
                text=dictlist[i].get(k)
                ndictlist[i].pop(k)
                ndictlist[i].update({f'{k}_org':text})
                clean=basic_clean(text,regexfilter=regexfilter)
                ndictlist[i].update({f'{k}_cleaned':clean})
                lemmatized=lemmatizor(text,regexfilter)
                stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=stop_words_extend_reduce)
                ndictlist[i].update({f'{k}_lemmatized':stopfilteredlemitezed})
                stem=stemmed(text,regexfilter)
                stopfilteredstem=stopfilter(stem,stop_words_extend_reduce=stop_words_extend_reduce)
                ndictlist[i].update({f'{k}_stemmed':stopfilteredstem})

    
 
                
               

    a=pd.DataFrame(ndictlist[0],columns=list(ndictlist[0].keys()),index=[0])
    for i,s in enumerate(ndictlist):
        b=pd.DataFrame(s,columns=list(s.keys()),index=[i])
        a=pd.merge(a,b,how='outer',right_index=False,left_index=False)
    df=a


  
       
    return df


In [3]:





lemmatized=lemmatizor(original,regexfilter=r'[^a-z0-9\'\s]')
print(lemmatized)
stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=["'","erdos","paul","a"])
print(stopfilteredlemitezed)
    

    

paul erdos and george polya were influential hungarian mathematicians who contributed a lot to the field erdos ' s name contains the hungarian letter ' o ' ' o ' with double acute accent but is often incorrectly written as erdos or erdos either by mistake or out of typographical necessity
george polya influential hungarian mathematicians contributed a lot field name contains hungarian letter double acute accent often incorrectly written either mistake typographical necessity


In [4]:
url = 'https://inshorts.com/en/read'
arts = get_all_shorts(url)




/Users/richardmacken/codeup-data-science/natural-language-processing-exercises/acquire.py:42: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 42 of the file /Users/richardmacken/codeup-data-science/natural-language-processing-exercises/acquire.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = soupify(get(base_url).content)


<Response [200]>
cat articles length:  12
length of all_articles:  12


/Users/richardmacken/codeup-data-science/natural-language-processing-exercises/acquire.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file /Users/richardmacken/codeup-data-science/natural-language-processing-exercises/acquire.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  cat_soup = soupify(get(cat_url).content)


<Response [200]>
cat articles length:  25
length of all_articles:  37
<Response [200]>
cat articles length:  25
length of all_articles:  62
<Response [200]>
cat articles length:  25
length of all_articles:  87
<Response [200]>
cat articles length:  24
length of all_articles:  111
<Response [200]>
cat articles length:  25
length of all_articles:  136
<Response [200]>
cat articles length:  25
length of all_articles:  161
<Response [200]>
cat articles length:  25
length of all_articles:  186
<Response [200]>
cat articles length:  25
length of all_articles:  211
<Response [200]>
cat articles length:  25
length of all_articles:  236
<Response [200]>
cat articles length:  25
length of all_articles:  261
<Response [200]>
cat articles length:  24
length of all_articles:  285


In [5]:

all_articles=deepcopy(arts)

In [6]:


stopped_lemma=dictlist_lemmatizor_else_stemmer(all_articles,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"],lemmatize=True)
stopped_stemmed=dictlist_lemmatizor_else_stemmer(all_articles,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"],lemmatize=False)




In [7]:
news_df=dictlist_super_NLP_comp(all_articles,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"],interestingkeys=['body']);news_df.T







,0,1,2,3,4,5,6,7,8,9,...,275,276,277,278,279,280,281,282,283,284
title,"Nigerian weightlifter in dope net, India may gain","Samsung launches Galaxy Star 2 Plus at Rs.7,335",India beat NZ 3-2 to enter CWG hockey finals,"Afghanistan wins SAFF title, spoils India's ha...","AAP drops Rajouri Garden candidate, a week bef...",Kashmir's famous Dal Lake freezes,"Indian Navy gets VLF, easy communication with ...",Bharti Airtel rakes in 61% profit,India's first Billiards Premier League,Infosys Gifts Sikka Shares Worth Rs 8.2cr,...,TVS Motor beats Hero MotoCorp to become 6th mo...,Car rental startup Zoomcar may go public via S...,Passenger vehicle wholesales rise by 92% in Se...,UP govt announces new EV policy to attract ₹30...,Amazon-backed Rivian's shares fall 9% after it...,Vintage cars on display to promote wildlife pr...,Vehicle registrations during festivals doubled...,"Honda Motor, LG to build $4.4 billion EV batte...",Mercedes-Benz sees 28% rise in sales in India ...,K'taka HC allows cab aggregators to operate au...
category,india,india,india,india,india,india,india,india,india,india,...,automobile,automobile,automobile,automobile,automobile,automobile,automobile,automobile,automobile,automobile
body_org,India may move up after Nigerian weightlifter ...,Samsung has unveiled the Galaxy start 2 Plus s...,In the CWG men's hockey semi-final against New...,Afghanistan won their maiden-SAFF Football Cha...,"Only a week before Delhi Assembly polls, Aam A...",After the recent snowfall in upper reaches of ...,The Indian navy has a new communication system...,"Bharti Airtel, India's top telecommunications ...",The Billiards and Snooker Association of Mahar...,"In a regulatory filing to the BSE on Friday, I...",...,TVS Motor Company Limited has become the sixth...,Bengaluru-based car rental startup Zoomcar has...,Passenger vehicle wholesales in India surged b...,The Uttar Pradesh government announced a new e...,Amazon-backed EV-maker Rivian on Friday recall...,"To create awareness about wildlife week, the K...",Vehicle registrations more than doubled in thi...,Honda Motor and LG Energy Solution on Tuesday ...,Mercedes-Benz India has registered 28% rise in...,The Karnataka High Court has allowed app-based...
body_cleaned,india may move up after nigerian weightlifter ...,samsung has unveiled the galaxy start 2 plus s...,in the cwg men's hockey semifinal against new ...,afghanistan won their maidensaff football cham...,only a week before delhi assembly polls aam aa...,after the recent snowfall in upper reaches of ...,the indian navy has a new communication system...,bharti airtel india's top telecommunications c...,the billiards and snooker association of mahar...,in a regulatory filing to the bse on friday in...,...,tvs motor company limited has become the sixth...,bengalurubased car rental startup zoomcar has ...,passenger vehicle wholesales in india surged b...,the uttar pradesh government announced a new e...,amazonbacked evmaker rivian on friday recalled...,to create awareness about wildlife week the ka...,vehicle registrations more than doubled in thi...,honda motor and lg energy solution on tuesday ...,mercedesbenz india has registered 28 rise in s...,the karnataka high court has allowed appbased ...
body_lemmatized,india may move nigerian weightlifter chika ama...,samsung unveiled galaxy start 2 plus smartphon...,cwg men hockey semifinal new zealand saturday ...,afghanistan maidensaff football championship d...,week delhi assembly polls aam aadmi party tues...,recent snowfall upper reaches kashmir himalaya...,indian navy new communication system critical ...,bharti airtel india top telecommunications com...,billiards snooker association maharashtrabsam ...,regulatory filing bse friday infosys ltd decid...,...,tvs motor company limited become sixth mostval...,bengalurubased car rental startup zoomcar repo...,passenger vehicle wholesales india surged 92 3...,uttar pradesh government announced new electri...,amazonbacked evmaker rivian fr

Exercises

The end result of this exercise should be a file named prepare.py that defines the requested functions.

In this exercise we will be defining some functions to prepare textual data. These functions should apply equally well to both the codeup blog articles and the news articles that were previously acquired.

    Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:
        Lowercase everything
        Normalize unicode characters
        Replace anything that is not a letter, number, whitespace or a single quote.

    Define a function named tokenize. It should take in a string and tokenize all the words in the string.

    Define a function named stem. It should accept some text and return the text after applying stemming to all the words.

    Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.

    Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

    This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.

    Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.

    Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.

    For each dataframe, produce the following columns:
        title to hold the title
        original to hold the original article/post content
        clean to hold the normalized and tokenized original with the stopwords removed.
        stemmed to hold the stemmed version of the cleaned data.
        lemmatized to hold the lemmatized version of the cleaned data.

    Ask yourself:
        If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
        lemmatized
        
        If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
        lemmatized

        If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?
        Stemmed unless the value added was greater enoungh to be account for the additional costs.


In [28]:
datadf=pd.read_json('data.json')

len(datadf)

df_C=datadf[datadf['language']=='C'];df_C.head(n=30)


df_Python=datadf[datadf['language']=='Python'];df_Python.head(n=30)


df_Java=datadf[datadf['language']=='Java'];df_Java.head(n=30)





all_articles=(datadf['readme_contents']).to_dict()
all_articles.keys()


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [71]:

def dictlist_super_NLP_comp(dictlist,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"]):
    ''
    ndictlist=deepcopy(dictlist)
    mapper=[]
    interestingkeys=list(ndictlist.keys())
    for i in range(0,len(ndictlist)):           
            k=interestingkeys[i]
            text=ndictlist.get(k)         
            org={f'org':text}
            clean=basic_clean(text,regexfilter=regexfilter)
            cleaned=({f'cleaned':clean})
            lemmatized=lemmatizor(text,regexfilter)
            stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=stop_words_extend_reduce)
            lemma={f'lemmatized':stopfilteredlemitezed}
            stem=stemmed(text,regexfilter)
            stopfilteredstem=stopfilter(stem,stop_words_extend_reduce=stop_words_extend_reduce)
            stemma={f'stemmed':stopfilteredstem}
            mapper.append({k:dict(**org,**cleaned,**lemma,**stemma)})

    
 
                
               


  
       
    return mapper


In [85]:
newdf=dictlist_super_NLP_comp(all_articles)


In [86]:

a=pd.DataFrame(newdf[0]).T
for i in range(1,len(newdf)):
    b=pd.DataFrame(newdf[i]).T
    a=pd.merge(a,b,how='outer',right_index=False,left_index=False)


    


In [91]:
df=pd.merge(datadf,a,how='left',right_index=True,left_index=True)
display(df.head(),datadf.head())## Verify merge


,repo,language,readme_contents,cleaned,lemmatized,org,stemmed
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\nview rendere...,let build simple database view rendered tutori...,# Let's Build a Simple Database\n\n[View rende...,let build simple database view rendered tutori...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\nthe old master ...,chibicc small c compiler old master moved hist...,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc small c compiler old master moved hist...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for reptying programs\n\n\nrept...,reptyr tool reptying programs reptyr utility t...,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr tool reptying programs reptyr utility t...
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\nbuildhttpsflatbadgennetgith...,introduction buildhttpsflatbadgennetgithubstat...,\n# Introduction\n\n[![Build](https://flat.bad...,introduction buildhttpsflatbadgennetgithubstat...


,repo,language,readme_contents
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===..."
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...


In [92]:
datadf=df #finish prepped


In [103]:

reorder=['language',
'repo',
 'readme_contents',
 'cleaned',
 'stemmed',
 'lemmatized']
  
datadf=datadf[reorder]


datadf




,language,repo,readme_contents,cleaned,stemmed,lemmatized
0,C,cstack/db_tutorial,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\nview rendere...,let build simple database view rendered tutori...,let build simple database view rendered tutori...
1,C,rui314/chibicc,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\nthe old master ...,chibicc small c compiler old master moved hist...,chibicc small c compiler old master moved hist...
2,C,nelhage/reptyr,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for reptying programs\n\n\nrept...,reptyr tool reptying programs reptyr utility t...,reptyr tool reptying programs reptyr utility t...
3,C,EyalAr/lwip,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...
4,C,ibireme/yyjson,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\nbuildhttpsflatbadgennetgith...,introduction buildhttpsflatbadgennetgithubstat...,introduction buildhttpsflatbadgennetgithubstat...
...,...,...,...,...,...,...
265,Java,Amterson/AlginProject,\n### 1.非中文单词不够一行会自动截断，用符号“-”连接起来；\n\n### 2.适配...,\n 1\n\n 2textviewandroidgravityandroidtextali...,1 2textviewandroidgravityandroidtextalignmentg...,1 2textviewandroidgravityandroidtextalignmentg...
266,Java,twitch4j/twitch4j,"<p align=""center""><a href=""https://twitch4j.gi...",p aligncentera hrefhttpstwitch4jgithubioimg sr...,p aligncentera hrefhttpstwitch4jgithubioimg sr...,p aligncentera hrefhttpstwitch4jgithubioimg sr...
267,Java,ittianyu/POCenter,## 外包集中营 ##\n\n整合多个软件外包平台项目信息，替你筛选优质项目\n\n![MI...,\n\n\n\nmit licensehttpsimgshieldsiogithubli...,mit licensehttpsimgshieldsiogithublicensemasha...,mit licensehttpsimgshieldsiogithublicensemasha...
268,Java,acloudyh/springCloud,## springCloud 学习\n\n[springCloud-config配置中心](...,springcloud \n\nspringcloudconfighttpsgithubc...,springcloud springcloudconfighttpsgithubcomacl...,springcloud springcloudconfighttpsgithubcomacl...
